# Deep Q Learning pseudo code
### 1. Q Value의 파라미터 초기화
### 2. Agent의 State(s) 초기화
### 3. 에피소드가 끝날 때까지 다음 (A~E) 반복
####     A.Q Value에 대한 E-greedy를 이용한 Action(a) 선택(Select_Action)
####     B.a를 실행하여 r(Reward)과 s'을 관측
####     C.s'에서 Q Value에 대한 greedy를 이용하여 a'선택(TD-Target)
####     D.Value 업데이트(미분은 파이토치, 텐서플로우와 같은 라이브러리가 대신 해주므로 손실 함수는 설정)
####     E.s<-s'
### 4. 에피소드가 끝나면 다시 2번으로 돌아가서 Value가 수렴할 때까진 반복

In [34]:
import gym
import collections #리플레이 버퍼 - 데이터의 사용자가 지정한 갯수만큼 최신 데이터로만 보유해주는 라이브러리(선입선출)
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [35]:
#하이퍼 파라미터
learning_rate = 0.0005
gamma = 0.98
buffer_limit = 50000 #CartPole은 비교적 단순한 문제로 한도를 5만으로 감소
batch_size = 32 #하나의 mini_batch 안에 32개의 데이터가 쓰임

In [36]:
#리플레이 버퍼 Class
#최신 5만 개의 데이터를 들고 있다가 필요한 때마다 batch_size 만큼의 데이터를 뽑아서 제공해주는 것
class ReplayBuffer(): 
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)

    #데터를 버퍼에 넣어주는 함수
    def put(self, transition): 
        self.buffer.append(transition)

    #랜덤하게 32개의 데이터를 뽑아 미니배치를 구성해주는 함수
    #done_mask는 종료 상태의 value를 마스킹해주기 위한 변수(종료 상태의 Value를 0으로 만들어 줌)
    #32개의 데이터는 각기 종류에 맞게 (s는 s끼리) 요소별로 모아서 pytorch의 텐서로 변환해주는 작업을 진행
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []

        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch. float), 
        torch.tensor(a_lst), torch.tensor(r_lst), 
        torch.tensor(s_prime_lst, dtype=torch. float), 
        torch.tensor(done_mask_lst)

    def size(self):
        return len(self.buffer)                                   

In [37]:
#Q_value network class
#nn.module 클래스를 상속받아 선언됌 - 뉴럴넷을 만들때 뼈대가 되는 클래스
#Q 네트워크의 파라미터들을 타킷 네트워크로 복사할 때 load_state_dict() 함수를 사용해 한 줄만으로 구현이 가능
class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 1)
        else:
            return out.argmax().item() #Q값이 제일 큰 액션을 선택

In [38]:
#학습 함수 - 에피소드 하나가 끝날 때마다 호출
def train(q, q_target, memory, optimizer):
    for i in range(10): #한 번 호출할 때마다 10개의 미니 배치를 뽑아 총 10번 업데이트하도록 구현 (한 에피소드에 총 320개의 데이터를 뽑아 사용)
        s, a, r, s_prime, done_mask = memory.sample(batch_size)

        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime). max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)

        optimizer.zero_grad()
        loss.backword() #역전파 
        optimizer.step()

In [39]:
#main 함수
def main():
    env = gym.make('CartPole-v1')
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict()) #q 네트워크의 파라미터의 값들을 q_target으로 전부 복사
    memory = ReplayBuffer()

    print_interval = 20
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)

    for n_epi in range(10000): #에피소드 진행 수
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200))
        #Decaying epsilon
        s = env.reset()
        done = False

        while not done:
            a = q.sample_action(torch.from_numpy(s). float (), epsilon)
            s_prime, r, done, info = env.step(a)
            done_mask = 0.0 if done else 1.0
            memory.put((s,a,r/100.0, s_prime, done_mask)) #보상을 적절하게 주기 위해서 100으로 나누어 줌
            s = s_prime
            score += r
            if done:
                break
        
        if memory.size()>2000: #데이터가 많이 쌓이지 않은 초기 상태에서 학습을 진행 시 학습이 치우치는 걸 방지하기 위해 2000개를 넘었을 때부터 학습 진행
            train(q, q_target, memory, optimizer)

        if n_epi%print_interval==0 and n_epi != 0:
            q_target.load_state_dict(q.state_dict())
            print("n_episode : {}, score : {:.1f}, n_butter : {}, eps : {:.1f}%".format (n_epi, score/print_interval, memory.size(), epsilon*100))
            score = 0.0

    env.close()

In [41]:
main()

n_episode : 20, score : 14.1, n_butter : 282, eps : 7.9%
n_episode : 40, score : 13.4, n_butter : 550, eps : 7.8%
n_episode : 60, score : 14.0, n_butter : 830, eps : 7.7%
n_episode : 80, score : 13.5, n_butter : 1100, eps : 7.6%
n_episode : 100, score : 13.3, n_butter : 1366, eps : 7.5%
n_episode : 120, score : 13.4, n_butter : 1634, eps : 7.4%
n_episode : 140, score : 13.3, n_butter : 1901, eps : 7.3%


ValueError: not enough values to unpack (expected 5, got 1)